In [12]:
#import things that may be useful
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
%matplotlib inline

In [5]:
#pull in data
column_defs = pd.read_csv('/Users/nick/w266_nlp_project/data/table_col_definitions.csv', encoding = 'cp1252')
ticket_data = pd.read_csv('/Users/nick/w266_nlp_project/data/ticket_data_no_autoreports.csv', encoding = 'cp1252')#,error_bad_lines=False)#,dtype = str)

In [6]:
column_defs

,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,NUMERIC_PRECISION
0,BugActions,BugActionID,1,NaN,NO,int,NaN,10.0
1,BugActions,BugID,2,NaN,NO,int,NaN,10.0
2,BugActions,UserID,3,NaN,NO,varchar,50.0,NaN
3,BugActions,ActionTime,4,NaN,NO,datetime,NaN,NaN
4,BugActions,AssignedID,5,NaN,NO,varchar,50.0,NaN
...,...,...,...,...,...,...,...,...
69,KBArticles,ProductID,8,((-1)),NO,int,NaN,10.0
70,KBArticles,Active,9,((1)),NO,int,NaN,10.0
71,KBArticles,UserFriendlyDesc,10,(''),NO,nvarchar,100.0,NaN
72,KBArticles,SalesProductGroupID,11,((0)),NO,int,NaN,10.0


In [89]:
ticket_data

,BugActionID,BugID,ActionUserID,ActionTime,AssignedID,Notes,StatusID,PriorityID,BAContactID,UpdateType,...,KBTitle,NumUses,NumTimesEscalated,DefaultPriorityID,PackageID,KBProductID,Active,UserFriendlyDesc,SalesProductGroupID,DefaultClassification
0,2119447,321438,tdoyens,2017-01-03 09:04:28.100000000,Support,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,Closed,Normal,40330,Worked,...,YEF: Status of E-File Submission,0,0,Normal,0,0,1,Status of E-File Submission,4,Questions
1,2119462,321439,kmcclung,2017-01-03 09:09:40.860000000,Support,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,Closed,Normal,37655,Worked,...,TFC: Create Year-End Forms - General Questions,0,0,Normal,-1,7,1,Tax Filing Center: Create Year-End Forms - Gen...,1,Questions
2,2119582,321441,dwood,2017-01-03 09:37:05.667000000,Support,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,Closed,Normal,146213,Worked,...,GO/GOES: Questions About Employee Sign-In,0,0,Normal,-1,43,1,GreenshadesOnline: Questions About Employee Si...,21,Questions
3,2119519,321442,tdoyens,2017-01-03 09:21:47.007000000,Support,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,Closed,Normal,133298,Worked,...,Benefits: Connector doesn't sync Benefits corr...,0,0,Normal,0,61,1,Benefits: Connector Doesn't Sync,40,NaN
4,2121069,321443,aowen,2017-01-03 15:30:33.723000000,PTS Development,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,In Progress,Normal,137930,Redirected,...,EFP: Payment failed or rejected due to invalid...,0,0,Normal,-1,40,1,NaN,30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6895,2224181,334486,wwilson,2017-01-30 21:00:34.497000000,Support,{\rtf1\fbidis\ansi\ansicpg1252\deff0\deflang10...,Blocked By Client,Normal,36410,Blocked,...,YEF: Form import fails or errors,0,0,Normal,0,0,1,Year-End Forms: Form import fails or errors,4,NaN
6896,2224185,334487,wwilson,2017-01-30 21:06:15.860000000,wwilson,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,Blocked By Client,Normal,148888,Blocked,...,Undetermined/Unknown Issue with unknown product,0,0,Normal,-1,-1,1,NaN,-1,NaN
6897,2225116,334494,jyoung,2017-01-31 09:59:32.600000000,Support,{\rtf1\fbidis\ansi\ansicpg1252\deff0\deflang10...,Closed,Normal,147124,Worked,...,EFP: File Acknowledgment Failed (YEF),0,0,Normal,0,0,1,NaN,4,NaN
6898,2225183,334499,jyoung,2017-01-31 10:01:23.347000000,Support,{\rtf1\fbidis\ansi\ansicpg1252\deff0\deflang10...,Closed,Normal,157886,Worked,...,EFP: File Acknowledgment Failed (TFC),0,0,Normal,0,0,1,NaN,1,NaN


### Sample Note

In [116]:
ticket_data.Notes[7]

'{\\rtf1\\fbidis\\ansi\\ansicpg1252\\deff0\\deflang1033{\\fonttbl{\\f0\\fnil\\fcharset0 Microsoft Sans Serif;}{\\f1\\froman\\fprq2\\fcharset0 Times New Roman;}}[cr][lf]{\\colortbl ;\\red31\\green73\\blue125;\\red56\\green86\\blue35;\\red5\\green99\\blue193;}[cr][lf]\\viewkind4\\uc1\\pard\\ltrpar\\f0\\fs24 client is only registered for 1099 on gc so i am going to tell them to use connector\\par[cr][lf]\\par[cr][lf]______________________\\par[cr][lf]1/03/17\\par[cr][lf]\\par[cr][lf]\\pard\\ltrpar\\cf1\\f1\\fs22 Hi Kordell,\\par[cr][lf]\\par[cr][lf]I see you are only registered for 1099\\rquote s with the Greenshades center so for w2\\rquote s you want to use the connector.  Please let me know if that clarifies everything, or if you have any more questions.\\par[cr][lf]\\par[cr][lf]Thank you,\\par[cr][lf]\\par[cr][lf]\\cf2\\b Will DiGiovanni\\par[cr][lf]Greenshades Tech Support\\par[cr][lf]888-255-3815 ext 1\\par[cr][lf]\\cf0\\b0\\f0\\fs24\\cf1\\f1\\fs22\\par[cr][lf]\\par[cr][lf]\\par[cr]

### First attempt at a cleanup function
This function removes most of the RTF syntax, but it is not perfect. Some internal RTF syntax remains.

In [117]:
def cleanup_note(note):
    #strip bulk of rtf formatting
    note = re.sub("[\{].*?[\}]", "",note)
    
    #remove rtf formatting newlines
    note = note.replace("}[cr][lf]", "")
    note = note.replace("[cr][lf]}", "")
    
    #remove leading rtf newlines
    if note[0:8] == '[cr][lf]':
        note = note[8:]
    
    #replace content-important newlines
    note = note.replace("\\par[cr][lf]", "\n")
    
    
    #remove leading rtf info if it is still there
    if note[0] == '\\':
        #first_backwack=note.find("\\") # get the position of first \
        ending_space=note.find(" ") # get the position of first space following the \
        note = note.replace(note[0:ending_space+1], '')
    
    return note

print(cleanup_note(ticket_data.Notes[7]))

client is only registered for 1099 on gc so i am going to tell them to use connector

______________________
1/03/17

\pard\ltrpar\cf1\f1\fs22 Hi Kordell,

I see you are only registered for 1099\rquote s with the Greenshades center so for w2\rquote s you want to use the connector.  Please let me know if that clarifies everything, or if you have any more questions.

Thank you,

\cf2\b Will DiGiovanni
Greenshades Tech Support
888-255-3815 ext 1
\cf0\b0\f0\fs24\cf1\f1\fs22


\cf0\b From:\b0  Kordell Bauer [\cf3\ul <mailto:kordell@hitinc.org>\cf0\ulnone ] \line\b Sent:\b0  Monday, January 2, 2017 9:58 PM\line\b To:\b0  Greenshades Support <\cf3\ul support@greenshades.com <mailto:support@greenshades.com>\cf0\ulnone >\line\b Subject:\b0  w-2s

Do I upload my w-2 forms through the connector or the Greenshades center  need to know asap thanks

\pard\ltrpar\f0\fs24



### StackOverflow RTF Stripping function
Direct link: https://gist.github.com/gilsondev/7c1d2d753ddb522e7bc22511cfb08676  
StackOverflow Post: https://stackoverflow.com/a/188877  
This appears to work much better than the function I created.

In [111]:
def striprtf(text):
    '''
    This function is from:
    https://gist.github.com/gilsondev/7c1d2d753ddb522e7bc22511cfb08676
    Found via StackOverflow:
    https://stackoverflow.com/a/188877
    '''
    
    pattern = re.compile(r"\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)", re.I)
   # control words which specify a "destionation".
    destinations = frozenset((
      'aftncn','aftnsep','aftnsepc','annotation','atnauthor','atndate','atnicn','atnid',
      'atnparent','atnref','atntime','atrfend','atrfstart','author','background',
      'bkmkend','bkmkstart','blipuid','buptim','category','colorschememapping',
      'colortbl','comment','company','creatim','datafield','datastore','defchp','defpap',
      'do','doccomm','docvar','dptxbxtext','ebcend','ebcstart','factoidname','falt',
      'fchars','ffdeftext','ffentrymcr','ffexitmcr','ffformat','ffhelptext','ffl',
      'ffname','ffstattext','field','file','filetbl','fldinst','fldrslt','fldtype',
      'fname','fontemb','fontfile','fonttbl','footer','footerf','footerl','footerr',
      'footnote','formfield','ftncn','ftnsep','ftnsepc','g','generator','gridtbl',
      'header','headerf','headerl','headerr','hl','hlfr','hlinkbase','hlloc','hlsrc',
      'hsv','htmltag','info','keycode','keywords','latentstyles','lchars','levelnumbers',
      'leveltext','lfolevel','linkval','list','listlevel','listname','listoverride',
      'listoverridetable','listpicture','liststylename','listtable','listtext',
      'lsdlockedexcept','macc','maccPr','mailmerge','maln','malnScr','manager','margPr',
      'mbar','mbarPr','mbaseJc','mbegChr','mborderBox','mborderBoxPr','mbox','mboxPr',
      'mchr','mcount','mctrlPr','md','mdeg','mdegHide','mden','mdiff','mdPr','me',
      'mendChr','meqArr','meqArrPr','mf','mfName','mfPr','mfunc','mfuncPr','mgroupChr',
      'mgroupChrPr','mgrow','mhideBot','mhideLeft','mhideRight','mhideTop','mhtmltag',
      'mlim','mlimloc','mlimlow','mlimlowPr','mlimupp','mlimuppPr','mm','mmaddfieldname',
      'mmath','mmathPict','mmathPr','mmaxdist','mmc','mmcJc','mmconnectstr',
      'mmconnectstrdata','mmcPr','mmcs','mmdatasource','mmheadersource','mmmailsubject',
      'mmodso','mmodsofilter','mmodsofldmpdata','mmodsomappedname','mmodsoname',
      'mmodsorecipdata','mmodsosort','mmodsosrc','mmodsotable','mmodsoudl',
      'mmodsoudldata','mmodsouniquetag','mmPr','mmquery','mmr','mnary','mnaryPr',
      'mnoBreak','mnum','mobjDist','moMath','moMathPara','moMathParaPr','mopEmu',
      'mphant','mphantPr','mplcHide','mpos','mr','mrad','mradPr','mrPr','msepChr',
      'mshow','mshp','msPre','msPrePr','msSub','msSubPr','msSubSup','msSubSupPr','msSup',
      'msSupPr','mstrikeBLTR','mstrikeH','mstrikeTLBR','mstrikeV','msub','msubHide',
      'msup','msupHide','mtransp','mtype','mvertJc','mvfmf','mvfml','mvtof','mvtol',
      'mzeroAsc','mzeroDesc','mzeroWid','nesttableprops','nextfile','nonesttables',
      'objalias','objclass','objdata','object','objname','objsect','objtime','oldcprops',
      'oldpprops','oldsprops','oldtprops','oleclsid','operator','panose','password',
      'passwordhash','pgp','pgptbl','picprop','pict','pn','pnseclvl','pntext','pntxta',
      'pntxtb','printim','private','propname','protend','protstart','protusertbl','pxe',
      'result','revtbl','revtim','rsidtbl','rxe','shp','shpgrp','shpinst',
      'shppict','shprslt','shptxt','sn','sp','staticval','stylesheet','subject','sv',
      'svb','tc','template','themedata','title','txe','ud','upr','userprops',
      'wgrffmtfilter','windowcaption','writereservation','writereservhash','xe','xform',
      'xmlattrname','xmlattrvalue','xmlclose','xmlname','xmlnstbl',
      'xmlopen',
   ))
   # Translation of some special characters.
    specialchars = {
      'par': '\n',
      'sect': '\n\n',
      'page': '\n\n',
      'line': '\n',
      'tab': '\t',
      'emdash': '\u2014',
      'endash': '\u2013',
      'emspace': '\u2003',
      'enspace': '\u2002',
      'qmspace': '\u2005',
      'bullet': '\u2022',
      'lquote': '\u2018',
      'rquote': '\u2019',
      'ldblquote': '\201C',
      'rdblquote': '\u201D',
   }
    stack = []
    ignorable = False       # Whether this group (and all inside it) are "ignorable".
    ucskip = 1              # Number of ASCII characters to skip after a unicode character.
    curskip = 0             # Number of ASCII characters left to skip
    out = []                # Output buffer.
    for match in pattern.finditer(text):
        word,arg,hex,char,brace,tchar = match.groups()
        if brace:
            curskip = 0
            if brace == '{':
                # Push state
                stack.append((ucskip,ignorable))
            elif brace == '}':
                # Pop state
                ucskip,ignorable = stack.pop()
        elif char: # \x (not a letter)
            curskip = 0
            if char == '~':
                if not ignorable:
                    out.append('\xA0')
            elif char in '{}\\':
                if not ignorable:
                    out.append(char)
            elif char == '*':
                ignorable = True
        elif word: # \foo
            curskip = 0
            if word in destinations:
                ignorable = True
            elif ignorable:
                pass
            elif word in specialchars:
                out.append(specialchars[word])
            elif word == 'uc':
                ucskip = int(arg)
            elif word == 'u':
                c = int(arg)
                if c < 0: c += 0x10000
                if c > 127: out.append(chr(c)) #NOQA
                else: out.append(chr(c))
                curskip = ucskip
        elif hex: # \'xx    
            if curskip > 0:
                curskip -= 1
            elif not ignorable:
                c = int(hex,16)
                if c > 127: out.append(chr(c)) #NOQA
                else: out.append(chr(c))
        elif tchar: #********
            if curskip > 0:
                curskip -= 1
            elif not ignorable:
                out.append(tchar)
    return ''.join(out)

In [118]:
#All the RTF syntax is removed and deleted extraneous newlines 
#(RTF parser autoreplaces the RTF linebreaks so [cr][fr] indicators may not be useful)
print(striprtf(ticket_data.Notes[7]).replace('[cr][lf]',''))

client is only registered for 1099 on gc so i am going to tell them to use connector

______________________
1/03/17

Hi Kordell,

I see you are only registered for 1099’s with the Greenshades center so for w2’s you want to use the connector.  Please let me know if that clarifies everything, or if you have any more questions.

Thank you,

Will DiGiovanni
Greenshades Tech Support
888-255-3815 ext 1



From: Kordell Bauer [<mailto:kordell@hitinc.org>] 
Sent: Monday, January 2, 2017 9:58 PM
To: Greenshades Support <support@greenshades.com <mailto:support@greenshades.com>>
Subject: w-2s

Do I upload my w-2 forms through the connector or the Greenshades center  need to know asap thanks





### Data Cleaning TODOS:
* Remove names or replace with generics
* Remove email addresses or replace with generics
* Remove phone numbers or replace with generics
* Remove references to client companies